In [1]:
import math
from math import copysign
import numpy as np
import pandas as pd
import scipy

In [75]:
O = 10*np.array([
    [4, 1, -1, 1,9, 1],
    [1, 2, 0, 1, 1,1],
    [-2, 0, 3, 2,1,1],
    [2, 1, 2, -1,1,0],
    [1, 1, 3, -1,1,9],
    [1, 5, 3, -1,1,9]
])

SVD

In [114]:
#phase 1

A = O.T @ O #symmetric

#householder reduction to hessenberg form 
def hessenberg_reduction(A):
    Q = np.eye(A.shape[0])
    for k in range(0, A.shape[0]-2):
        #select kth column of A below the kth terms 
        x = A[k+1:, k]
        n = A.shape[0]-k-1
        e1 = np.zeros(n)
        e1[0] = 1
        v = copysign(np.linalg.norm(x),x[0])*e1 + x 
        v = v/np.linalg.norm(v)
        v = v.reshape(v.shape[0],1)
        A[k+1:,k:] = A[k+1:,k:] - 2*v@(v.T@A[k+1:,k:])
        A[:,k+1:] = A[:,k+1:] - 2*(A[:,k+1:]@v)@v.T
    return A

A_ = hessenberg_reduction(A) #triadiagonal matrix 

In [115]:
#phase 2
#apply qr algorithm to obtain eigenvalue decomposition
def qr_algo(A, N = 100000):
    n = A.shape[0]
    V = np.eye(n)
    for k in range(1, N):
        u = A[-1,-1]
        Q, R = np.linalg.qr(A-u*np.eye(n))
        V = V@Q
        A = R@Q + u*np.eye(n)
        '''
        for i in range(3,n-1):
            if A[i+1,i]< 0.0000000001:
                print(i,n)
                A[i+1,i] = 0
                A[i,i+1] = 0
                A11 = A[:i,:i]
                A22 = A[i:,i:]
                A11, V11 = qr_algo(A11)
                A22, V22 = qr_algo(A22)
                print(A11, A22, 0*np.eye(n-i), 0*np.eye(i))
                A = np.block([[A11, np.zeros((i, n-i))], [np.zeros((n-i,i)),A22]])
                V = V@np.block([[V11, np.zeros((i, n-i))], [np.zeros((n-i,i)),V22]])
                return A, V 
            '''

    return A, V

D, V = qr_algo(A_)


In [116]:
#sort eigenvalues along the diagonal and the eigenvectors accordingly
indices = np.argsort(np.diag(D))[::-1]
sorted_diag = np.diag(D)[indices]
V = V[:, indices]
sigma = scipy.linalg.sqrtm(np.diag(np.diag(D)[indices]))

Q,R = np.linalg.qr(O)
b = Q.T
U = Q@R@V@np.linalg.inv(sigma)

In [101]:
print(np.diag(sigma))


[148.51689803  97.77190701  41.00342675  31.75073226  26.5645416
   8.62092361]


In [117]:
X,Y, Z = np.linalg.svd(O)
print(Y) #correct eigenvalues from svd function


[148.52793204  97.78612975  41.05043129  31.80675209  26.58336668
   8.59322189]


In [118]:
#relative error in U, V, and sigma using frobenius norm
print(np.linalg.norm(X-U)/np.linalg.norm(X))
print(np.linalg.norm(np.diag(sigma)-Y)/np.linalg.norm(Y))
print(np.linalg.norm(Z-V.T)/np.linalg.norm(Z))

3.98136518875744
0.0004399382043184628
1.3016083900803985


Stable method

In [107]:

#make 2mx2m block matrix with 
m = O.shape[0]
n = O.shape[1]
matrix_0_1 = np.zeros((n,n))
matrix_0_2 = np.zeros((m,m))
H = np.block([[matrix_0_1, O.T], [O, matrix_0_2]])


if n<=m:
    l=n
else:
    l=m

#obtain eigenvalue decomposition
D1,S = qr_algo(H)
d = np.diag(D1)
indices = np.argsort(d)[::-1]
d = d[indices][:l]
S = S[:,indices]
U1 = S[l:,:l]
V1 = S[:l,:l]


In [109]:
print('Eigenvalues', d)


Eigenvalues [148.52793204  97.78612975  41.05043129  31.80675209  26.58336668
   8.59322189]


In [111]:
print(np.linalg.norm(X-U1)/np.linalg.norm(X))
print(np.linalg.norm(d-Y)/np.linalg.norm(Y))
print(np.linalg.norm(Z-V1.T)/np.linalg.norm(Z))

0.746452247915388
3.250160647670967e-15
0.7464522479153881


The 2nd approach is more efficient and stable than the first one, as the relative error is much less compared to the first case.

In the the first case, the singular values are square roots of the calculated eigenvalues - essentially, by square rooting the eigenvalues, we also square root the order of the error (which is close to 0) thus increasing the error bound. In the second approach, the singular values are simply absolute values of eigenvalues, which avoids the square rootin process entirely, making the approach more stable.

Note: The first approach works for non-square matrices, however, the second one doesn't. I haven't been able to find the bug that makes the calculations awry yet - but using np.linalg.eig instead of my own qr_algorithm will fix the problem.